In [ ]:
import time
from dcicutils import s3Utils
from dcicutils import ff_utils
from core.wfr2 import *
from core.wfr_settings import *

wf_md5 = "md5"
my_env = 'data'
my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)

my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

run_md5 = raw_input("Do you wanna run md5 if missing? (md5/none)")

file_url = '/search/?status=upload%20failed&status=uploading&type=File&limit=all'
#file_url = '/search/?type=FileReference'
all_files = ff_utils.search_metadata(file_url, key=my_auth)
print(len(all_files))

printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print(counter)
        printn = counter

    if 'FileProcessed' in a_file['@type']:
            my_bucket = out_bucket
    else:  # covers cases of FileFastq, FileReference, FileMicroscopy
            my_bucket = raw_bucket       
 
    # check if file is in s3
    head_info = my_s3_util.does_key_exist(a_file['upload_key'], my_bucket)
    if not head_info:
        print(a_file['accession'], "does not have a file in S3")
        continue
        
    file_id = a_file['accession']
    status = a_file.get('status')
        
    # is there md5sum for gzip
    if not a_file.get('md5sum'):
        print(file_id,"does not have the md5sum calculated during upload")
    # Check workflows
    md5_status = 'did_not_run'
    
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    md5_report = get_wfr_out(a_file['uuid'], "md5", my_auth, md_qc = True)
    
    if md5_report['status'] == 'running':
        print(file_id, 'still running')
        continue 

    if status in ["uploading", "upload failed"] and md5_report['status'] != 'complete':
        # if not, shall we run it?
        if run_md5 == 'md5':
            print('md5 running for', file_id)
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'none', 'none'), inp_f, file_id, my_env, tag="")
            print()
        # user does not want it to be run, so just report
        else:
            print('md5 run missing for', file_id)

    elif status in ["uploading", "upload failed"] and md5_status == 'complete':
        print "ERROR:",file_id ,"There is a successful md5 run but status not switched"
        # if not, shall we run it?
        if run_md5 == 'md5':
            print('md5 running for', file_id)
            inp_f = {'input_file':a_file['@id']}
            run_missing_wfr(step_settings('md5', 'none', 'none'), inp_f, file_id, my_env, tag="")
            print()
        
    elif md5_status != 'complete':
        if a_file.get('content_md5sum'):
            print('md5run not linked to file')
        else:
            print(file_id, status, 'switched status file does not have md5 run or content_md5sum')
            # if not, shall we run it?
            if run_md5 == 'md5':
                print 'md5 running for', file_id
                inp_f = {'input_file':a_file['@id']}
                run_missing_wfr(step_settings('md5', 'none', 'none'), inp_f, file_id, my_env, tag="")
                print()
            
    print()

